In [1]:
import pandas as pd

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    # path = '~/Documents/GitHub/f1-analytics/'
    path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

In [2]:
races = pd.read_csv(path+'data/races.csv')
qualifying = pd.read_csv(path+'data/qualifying.csv')
constructors = pd.read_csv(path+'data/constructor_standings.csv')
drivers = pd.read_csv(path+'data/driver_standings.csv')
results = pd.read_csv(path+'data/results.csv')

### Redundant until new weather collection method is implemented
# weather = pd.read_csv(path+'data/weather.csv')

In [14]:
df1 = pd.merge(races, results, how='inner', on=['season', 'round', 'circuit_id']).drop(['url','points', 'status', 'time'], axis = 1)
df2 = pd.merge(df1, drivers, how='left', on=['season', 'round', 'driver']) 
df3 = pd.merge(df2, constructors, how='left', on=['season', 'round', 'constructor'])

merged_df = pd.merge(df3, qualifying, how='inner', on=['season', 'round', 'grid']).drop(['driver_y', 'constructor_y'], axis=1)

merged_df.rename(columns={'driver_x': 'driver', 'constructor_x': 'constructor'}, inplace=True)

merged_df.shape

(3191, 26)

In [13]:
# fill/drop nulls

for col in ['driver_points', 'driver_wins', 'driver_standings_pos', 'constructor_points', 
            'constructor_wins' , 'constructor_standings_pos']:
    merged_df[col].fillna(0, inplace=True)
    merged_df[col] = merged_df[col].map(lambda x: int(x))
    
merged_df.dropna(inplace=True)

merged_df.head()

ValueError: could not convert string to float: 'DNF'

In [15]:
merged_df.query('season == 2014 & round == 1')

,season,round,circuit_id,country,lat,long,date,driver,date_of_birth,nationality,...,driver_standings_pos,constructor_points,constructor_wins,constructor_standings_pos,pos,final_time,q_delta,q1,q2,q3
0,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16 06:00:00,rosberg,1985-06-27,German,...,1,25.0,1,2,3,104.595,0.364,92.564,102.264,104.595
1,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16 06:00:00,kevin_magnussen,1992-10-05,Danish,...,2,33.0,0,1,4,105.745,1.514,90.949,103.247,105.745
2,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16 06:00:00,button,1980-01-19,British,...,3,33.0,0,1,11,104.437,0.206,91.396,104.437,NaN
3,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16 06:00:00,alonso,1981-07-29,Spanish,...,4,18.0,0,3,5,105.819,1.588,91.388,102.805,105.819
4,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16 06:00:00,bottas,1989-08-28,Finnish,...,5,10.0,0,4,10,108.147,3.916,91.601,103.852,108.147
5,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16 06:00:00,hulkenberg,1987-08-19,German,...,6,9.0,0,5,7,106.030,1.799,93.893,103.658,106.03
6,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16 06:00:00,raikkonen,1979-10-17,Finnish,...,7,18.0,0,3,12,104.494,0.263,92.439,104.494,NaN
7,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16 06:00:00,vergne,1990-04-25,French,...,8,6.0,0,6,6,105.864,1.633,93.488,103.849,105.864
8,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16 06:00:00,kvyat,1994-04-26,Russian,...,9,6.0,0,6,8,107.368,3.137,93.777,104.331,107.368
9,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16 06:00:00,perez,1990-01-26,Mexican,...,10,9.0,0,5,16,107.293,3.062,94.141,107.293,NaN


In [7]:
merged_df.to_csv(path+'data/merged.csv', index=False)